<a href="https://colab.research.google.com/github/trianggadios/uas-ai/blob/main/Testing_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Package and Mounting to Gdrive

In [ ]:
from google.colab import drive

from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.models import load_model

from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np

import pandas

import librosa
import librosa.feature
import librosa.display

import warnings

warnings.simplefilter("ignore")

In [ ]:
drive.mount('/content/gdrive')

%cd gdrive/MyDrive/Dataset

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/MyDrive/Dataset'
/content/gdrive/MyDrive/Dataset


# Load Model

In [ ]:
loaded_models = Sequential([
    Dense(100, input_dim=1000),
    Activation('relu'),
    Dense(2),
    Activation('softmax')
])

In [ ]:
loaded_models.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
loaded_models.load_weights('gender_voice_model_full_data_v1.h5')

# Testing with Data

In [ ]:
df_test = pandas.read_csv('suara/cv-corpus-6.1-2020-12-11/id/test.tsv', sep='\t')
df_test = df_test.drop(['client_id', 'sentence', 'up_votes', 'down_votes', 'age', 'accent', 'locale', 'segment'], axis=1)
df_test.dropna(subset=['gender'], inplace=True)
df_test_male = df_test[df_test['gender'] == 'male']
df_test_female = df_test[df_test['gender'] == 'female']
df_test = pandas.concat([df_test_male, df_test_female])
df_test.reset_index()
df_test['gender'].unique()

array(['male', 'female'], dtype=object)

In [ ]:
def extract_feature_song(audio_path):
  x, sr = librosa.load(audio_path)
  mfcc = librosa.feature.mfcc(x)
  mfcc = mfcc/np.amax(np.absolute(mfcc))
  return np.ndarray.flatten(mfcc)[:1000]

In [ ]:
for index, row in df_test.iterrows():
  if index == 2:
    break

  audio_path = f'suara/cv-corpus-6.1-2020-12-11/id/clips/{row["path"]}'
  features = extract_feature_song(audio_path)

  a = np.array([features])

  print(f'Test case: {index+1}')
  print(f'Test File: {row["path"]}')
  print(f'Test Label: {row["gender"]}')

  result_predict = loaded_models.predict(a)[0]
  print('### Model Predict ###')
  print(f'Female: {round(result_predict[0]*100, 5)}%')
  print(f'Male: {round(result_predict[1]*100, 5)}%')
  print('')

Test case: 11
Test File: common_voice_id_20953397.mp3
Test Label: male
### Model Predict ###
Female: 0.0%
Male: 100.0%

Test case: 45
Test File: common_voice_id_23748178.mp3
Test Label: male
### Model Predict ###
Female: 0.0%
Male: 100.0%

Test case: 46
Test File: common_voice_id_23748179.mp3
Test Label: male
### Model Predict ###
Female: 0.00011%
Male: 99.99989%

Test case: 47
Test File: common_voice_id_23748186.mp3
Test Label: male
### Model Predict ###
Female: 6e-05%
Male: 99.99994%

Test case: 121
Test File: common_voice_id_22794281.mp3
Test Label: male
### Model Predict ###
Female: 0.59865%
Male: 99.40135%

Test case: 122
Test File: common_voice_id_22794283.mp3
Test Label: male
### Model Predict ###
Female: 0.0%
Male: 100.0%

Test case: 123
Test File: common_voice_id_22794284.mp3
Test Label: male
### Model Predict ###
Female: 4.45509%
Male: 95.54491%

Test case: 124
Test File: common_voice_id_22794285.mp3
Test Label: male
### Model Predict ###
Female: 0.91287%
Male: 99.08714%

Tes

KeyboardInterrupt: ignored

# Testing with Voice

In [ ]:
# Code for recording audio from the browser
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output


class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.mp3','wb+') as f:
    f.write(b)
  return 'audio.mp3'

In [ ]:
import IPython.display as ipd

def synth():
  print("Now recording for 10 seconds, say what you will...")
  record(10)
  print("Generate audio")
  in_fpath = Path("audio.mp3")
  print("Audio recording complete")
InvokeButton('Start recording', synth)

Now recording for 10 seconds, say what you will...


<IPython.core.display.Javascript object>

Generate audio


In [ ]:
ipd.Audio("audio.mp3")

In [ ]:
def extract_feature_song(audio_path):
  x, sr = librosa.load(audio_path)
  mfcc = librosa.feature.mfcc(x)
  mfcc = mfcc/np.amax(np.absolute(mfcc))
  return np.ndarray.flatten(mfcc)[:1000]

In [ ]:
audio_path = f'audio.mp3'
features = extract_feature_song(audio_path)

a = np.array([features])

print(f'Test case: 1')
print(f'Test File: {audio_path}')
print(f'Test Label: Voice Record')

result_predict = loaded_models.predict(a)[0]
print('### Model Predict ###')
print(f'Female: {round(result_predict[0]*100, 5)}%')
print(f'Male: {round(result_predict[1]*100, 5)}%')
print('')

Test case: 1
Test File: audio.mp3
Test Label: Voice Record
### Model Predict ###
Female: 3e-05%
Male: 99.99996%

